In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Setup

In [2]:
!ls ../input/ndscindexes/ndscindexes
!ls ../input/ndscindexes/ndscindexes/trn_index/
!ls ../input/ndscindexes/ndscindexes/trn_index/face/

ls: cannot access '../input/ndscindexes/ndscindexes': No such file or directory
ls: cannot access '../input/ndscindexes/ndscindexes/trn_index/': No such file or directory
ls: cannot access '../input/ndscindexes/ndscindexes/trn_index/face/': No such file or directory


In [3]:
x_train = np.load("../input/ndscloaded/x_train.npy")
x_test = np.load("../input/ndscloaded/x_test.npy")
y_train = np.load("../input/ndscloaded/y_train.npy")
features = np.load("../input/ndscloaded/features.npy")
test_features = np.load("../input/ndscloaded/test_features.npy")
word_index = np.load("../input/ndscloaded/word_index.npy").item()
embedding_matrix = np.load("../input/ndscloaded/embedding_matrix.npy")

In [4]:
print(np.shape(x_train))
print(x_train)
print(np.shape(x_test))
print(x_test)
print(np.shape(y_train))
print(y_train)
print(np.shape(embedding_matrix))

(666615, 70)
[[   0    0    0 ...    5  135  126]
 [   0    0    0 ...  226  210  213]
 [   0    0    0 ...   15  893 1342]
 ...
 [   0    0    0 ...    2   10  297]
 [   0    0    0 ...    4   30   18]
 [   0    0    0 ...  132  106  265]]
(172402, 70)
[[   0    0    0 ...   16   37   69]
 [   0    0    0 ...   16    5   37]
 [   0    0    0 ...   37   99 1360]
 ...
 [   0    0    0 ...  211  158 1387]
 [   0    0    0 ...    8  222  313]
 [   0    0    0 ...   96    8   47]]
(666615,)
[ 2 15 18 ... 25 25  7]
(48862, 300)


In [5]:
batch_size = 128

indices = np.arange(len(x_train))
np.random.shuffle(indices)
i = 0
x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
x_train_batch = [[embedding_matrix[token] for token in title][::-1] for title in x_train_batch_tokens]
# [::-1] because I want it back in the correct order
y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
y_train_batch = np.zeros((batch_size, 58))
y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

print(indices)
print(x_train_batch_tokens)
print(np.shape(x_train_batch))
print(y_train_batch_targets)
print(y_train_batch)

print(type(indices))
print(type(x_train_batch_tokens))
print(type(np.shape(x_train_batch[:1])))
print(type(x_train_batch))
print(type(y_train_batch_targets))
print(type(y_train_batch))

[155302 385349 557733 ... 429814 234892  35168]
[[   0    0    0 ...    7  126   97]
 [   0    0    0 ...  185  567   51]
 [   0    0    0 ...  357  348  928]
 ...
 [   0    0    0 ...  244  216  115]
 [   0    0    0 ...  695  168   80]
 [   0    0    0 ...  159 1102   62]]
(128, 70, 300)
[ 3  1  9  3 32 20 22  4 18  5 18  7 29 34 25 32 33  3  2 25 37  5 25 13
 42 18  4  5 35 32 25 12 18  2 25 31 35 25 31 27  1 42 18 12  4 12 12 12
 20  3  1 42 12  2 20 30 35  7  7  4 22 14 34 20  3  4  4  1 26 42  4 38
 18  3 26  3 11  2 29  4 38 25  5 25  3 43 20  3 31  2  3  5 35 49 21  1
 31 31 32 32 32 35 18  5 22 14 18 32 41 26  4 35  2 11  5  3 31 21  3 31
  5  3  4 12 35 41  4 35]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'tuple'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [6]:
# Data providers
batch_size = 128

indices = np.arange(len(x_train)-1)
# print(len(x_train))
# print(np.random.shuffle())

def batch_gen():
    n_batches = math.floor(len(x_train) / batch_size)
    while True: 
        np.random.shuffle(indices)  # Shuffle the data.
        for i in range(n_batches):
            x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
            x_train_batch = [[embedding_matrix[token] for token in title][::-1] for title in x_train_batch_tokens]
            # [::-1] because I want it back in the correct order
            y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
            y_train_batch = np.zeros((batch_size, 58))
            y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1
            yield (np.array(x_train_batch), y_train_batch)

# Training

In [7]:
from keras.models import Sequential, Model
from keras.layers import Input, CuDNNLSTM, Dense, Bidirectional, Activation, Flatten, RepeatVector, Permute
from keras.layers import merge

In [8]:
from keras.models import Model
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Activation, Input, Dropout, Permute, Reshape, concatenate, Flatten
from keras.layers import BatchNormalization
from keras import regularizers

INPUT_DIM = 300
TIME_STEPS = 70
# if True, the attention vector is shared across the input_dimensions where the attention is applied.
SINGLE_ATTENTION_VECTOR = False

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = concatenate([inputs, a_probs])
    return output_attention_mul

inputs_embed = Input((TIME_STEPS, INPUT_DIM))

lstm_1 = CuDNNLSTM(64, return_sequences=True)(inputs_embed)
lstm_2 = CuDNNLSTM(64, return_sequences=True)(lstm_1)

attention = attention_3d_block(lstm_2)
attention = Flatten()(attention)

fc_layer = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(attention)
fc_layer = BatchNormalization()(fc_layer)
fc_layer = Dropout(0.2)(fc_layer)
outputs = Dense(58, activation='softmax')(fc_layer)

model = Model(inputs=inputs_embed, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
mg = batch_gen()

In [10]:
model.fit_generator(mg, epochs=5,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/5
1000/1000 [==============================] - 63s 63ms/step - loss: 1.7547 - acc: 0.5464
Epoch 2/5
1000/1000 [==============================] - 58s 58ms/step - loss: 1.3365 - acc: 0.6276
Epoch 3/5
1000/1000 [==============================] - 58s 58ms/step - loss: 1.2489 - acc: 0.6451
Epoch 4/5
1000/1000 [==============================] - 57s 57ms/step - loss: 1.2035 - acc: 0.6548
Epoch 5/5
1000/1000 [==============================] - 56s 56ms/step - loss: 1.1718 - acc: 0.6626


In [11]:
model.fit_generator(mg, epochs=5,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/5
1000/1000 [==============================] - 57s 57ms/step - loss: 1.1392 - acc: 0.6710
Epoch 2/5
1000/1000 [==============================] - 56s 56ms/step - loss: 1.1322 - acc: 0.6713
Epoch 3/5
1000/1000 [==============================] - 55s 55ms/step - loss: 1.1178 - acc: 0.6744
Epoch 4/5
1000/1000 [==============================] - 55s 55ms/step - loss: 1.1002 - acc: 0.6788
Epoch 5/5
1000/1000 [==============================] - 53s 53ms/step - loss: 1.0900 - acc: 0.6825


In [12]:
model.fit_generator(mg, epochs=5,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/5
1000/1000 [==============================] - 56s 56ms/step - loss: 1.0852 - acc: 0.6829
Epoch 2/5
1000/1000 [==============================] - 55s 55ms/step - loss: 1.0698 - acc: 0.6862
Epoch 3/5
1000/1000 [==============================] - 54s 54ms/step - loss: 1.0657 - acc: 0.6887
Epoch 4/5
1000/1000 [==============================] - 54s 54ms/step - loss: 1.0591 - acc: 0.6891
Epoch 5/5
1000/1000 [==============================] - 53s 53ms/step - loss: 1.0596 - acc: 0.6896


In [13]:
model.fit_generator(mg, epochs=5,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/5
1000/1000 [==============================] - 57s 57ms/step - loss: 1.0510 - acc: 0.6896
Epoch 2/5
1000/1000 [==============================] - 56s 56ms/step - loss: 1.0383 - acc: 0.6920
Epoch 3/5
1000/1000 [==============================] - 55s 55ms/step - loss: 1.0366 - acc: 0.6921
Epoch 4/5
1000/1000 [==============================] - 54s 54ms/step - loss: 1.0379 - acc: 0.6940
Epoch 5/5
1000/1000 [==============================] - 54s 54ms/step - loss: 1.0326 - acc: 0.6959


In [14]:
model.fit_generator(mg, epochs=5,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/5
1000/1000 [==============================] - 57s 57ms/step - loss: 1.0248 - acc: 0.6973
Epoch 2/5
1000/1000 [==============================] - 56s 56ms/step - loss: 1.0136 - acc: 0.7006
Epoch 3/5
1000/1000 [==============================] - 57s 57ms/step - loss: 1.0188 - acc: 0.6993
Epoch 4/5
1000/1000 [==============================] - 58s 58ms/step - loss: 1.0216 - acc: 0.6965
Epoch 5/5
1000/1000 [==============================] - 58s 58ms/step - loss: 1.0108 - acc: 0.7000


# Inference

In [15]:
# prediction part
batch_size = 128
def batch_gen():
    n_batches = math.ceil(len(x_test) / batch_size)
    for i in range(n_batches):
        x_test_batch_tokens = x_test[i*batch_size:(i+1)*batch_size]
        x_test_batch = [[embedding_matrix[token] for token in title][::-1] for title in x_test_batch_tokens]
        # [::-1] because I want it back in the correct order
        yield np.array(x_test_batch)

In [16]:
import time
start_time = time.time()
all_preds = []
for x in batch_gen():
    all_preds.extend(model.predict(x))
print(time.time()-start_time)

86.31438326835632


In [17]:
test_df = pd.read_csv("../input/ndsc-beginner/test.csv")
test_df["Supercategory"] = test_df["image_path"].str[0]
supercats = test_df["Supercategory"]
supercat_dict = {
    "b" : np.array([1]*17 + [0]*14 + [0]*27),
    "f" : np.array([0]*17 + [1]*14 + [0]*27),
    "m" : np.array([0]*17 + [0]*14 + [1]*27)
}

In [18]:
print(np.shape(all_preds))
y_te = [np.argmax(pred) for pred,supercat in zip(all_preds,supercats)]

submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission_no_cheat.csv", index=False)

(172402, 58)


In [19]:
print(np.shape(all_preds))
y_te = [np.argmax(pred*supercat_dict[supercat]) for pred,supercat in zip(all_preds,supercats)]

submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission.csv", index=False)

(172402, 58)


In [20]:
submit_df.sample(100)

,itemid,Category
168995,1745771370,45
163286,338592772,31
27191,1824853077,15
74502,111862480,4
34973,103549832,12
169954,1684534040,35
69212,1766720772,12
30669,1523996108,12
141403,629973453,41
47119,1734216044,13


In [21]:
submit_df

,itemid,Category
0,370855998,5
1,637234604,5
2,690282890,5
3,930913462,5
4,1039280071,5
5,1327710392,3
6,1328802799,4
7,1330468145,5
8,1677309730,5
9,1683142205,5
